In [78]:
import pandas as pd

In [79]:
pd.set_option('display.max_columns', None)

In [80]:
df = pd.read_parquet('data_features/audio_diarization.parquet')
df2 = pd.read_parquet('data_features/audio_features.parquet')
df = pd.merge(df, df2, how='left', left_on=['filename'], right_on=['applicationid'])

In [81]:
app_data = pd.read_excel('data_features/test_audio.xlsx', header=1)

In [82]:
df = pd.merge(df, app_data, how='left', left_on=['filename'], right_on=['Файлы'])

In [83]:
target_data = pd.read_parquet('data/train_target_data.parquet')

In [84]:
feature_cols = ['total_segments',
       'avg_segment_duration', 'std_segment_duration', 'max_speaker_duration',
       'min_speaker_duration', 'dominant_speaker_ratio', 'speaking_turns_diff',
       'speech_density', 'avg_pause_between_segments',
       'audio_feature_0', 'audio_feature_1', 'audio_feature_2',
       'audio_feature_3', 'audio_feature_4', 'audio_feature_5',
       'audio_feature_6', 'audio_feature_7', 'audio_feature_8',
       'audio_feature_9', 'audio_feature_10', 'audio_feature_11',
       'audio_feature_12', 'audio_feature_13', 'audio_feature_14',
       'audio_feature_15', 'audio_feature_16', 'audio_feature_17',
       'audio_feature_18', 'audio_feature_19', 'audio_feature_20',
       'audio_feature_21', 'audio_feature_22', 'audio_feature_23',
       'audio_feature_24', 'audio_feature_25', 'audio_feature_26',
       'audio_feature_27', 'audio_feature_28', 'audio_feature_29',
       'audio_feature_30', 'audio_feature_31', 'audio_feature_32',
       'audio_feature_33', 'audio_feature_34', 'audio_feature_35',
       'audio_feature_36', 'audio_feature_37', 'audio_feature_38',
       'audio_feature_39', 'audio_feature_40', 'audio_feature_41',
       'audio_feature_42', 'audio_feature_43']

In [85]:
df = df.query("credit_rule.notna()")

In [86]:
df = df.set_index(df['APPLICATIONID'])

In [87]:
X = df[feature_cols]
y = df['credit_rule']

In [88]:
from lightgbm import LGBMClassifier

In [89]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from  lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import precision_recall_curve, auc
from lightgbm import LGBMClassifier


pd.set_option('display.max_columns', None)

In [90]:
X.fillna(0, inplace=True)

/tmp/ipykernel_5318/2534066105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)


In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [92]:
model = LGBMClassifier(random_state=42, min_data_in_leaf=800, n_estimators=10)

In [93]:
model.fit(X_train, y_train)

[LightGBM] [Warning] min_data_in_leaf is set=800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=800
[LightGBM] [Warning] min_data_in_leaf is set=800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=800
[LightGBM] [Info] Number of positive: 3183, number of negative: 121
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000820 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12760
[LightGBM] [Info] Number of data points in the train set: 3304, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.963378 -> initscore=3.269789
[LightGBM] [Info] Start training from score 3.269789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,10
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [94]:
from sklearn.isotonic import IsotonicRegression

In [95]:
calib = IsotonicRegression()

In [96]:
pred = model.predict_proba(X_train)[:, 1]


[LightGBM] [Warning] min_data_in_leaf is set=800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=800


In [97]:
calib.fit(pred, y_train)

,y_min,None
,y_max,None
,increasing,True
,out_of_bounds,'nan'


In [98]:
def pr_auc(y, pred):
    precision, recall, _ = precision_recall_curve(y, pred)
    return auc(recall, precision)

In [99]:
pred = model.predict_proba(X_train)[:, 1]
pred = calib.predict(pred)
print(f"PRC_AUC: {pr_auc(y_train, pred)}")
print(f"GINI: {roc_auc_score(y_train, pred) * 2 - 1}")
X_train['pd'] = pred
train_answer = pd.DataFrame(X_train['pd']).reset_index()


[LightGBM] [Warning] min_data_in_leaf is set=800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=800
PRC_AUC: 0.9855758854405763
GINI: 0.4626359559955653


In [100]:
pred = model.predict_proba(X_test)[:, 1]
pred = calib.predict(pred)
print(f"PRC_AUC: {pr_auc(y_test, pred)}")
print(f"GINI: {roc_auc_score(y_test, pred) * 2 - 1}")
X_test['pd'] = pred
test_answer = pd.DataFrame(X_test['pd']).reset_index()


[LightGBM] [Warning] min_data_in_leaf is set=800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=800
PRC_AUC: 0.9879143646343969
GINI: 0.43365330321852036


In [101]:
df_test = pd.read_parquet('data_features/validation_audio_diarization.parquet')
df2_test = pd.read_parquet('data_features/validation_audio_features.parquet')
df_test = pd.merge(df_test, df2_test, how='left', left_on=['filename'], right_on=['applicationid'])

In [102]:
app_data = pd.read_excel('data_features/validation_audio.xlsx', header=1)

In [103]:
df_test = pd.merge(df_test, app_data, how='left', left_on=['filename'], right_on=['Файлы'])

In [104]:
df_test = df_test.query("credit_rule.notna()")

In [105]:
df_test.fillna(0, inplace=True)

In [106]:
pred = model.predict_proba(df_test[feature_cols])[:, 1]
pred = calib.predict(pred)
print(f"PRC_AUC: {pr_auc(df_test['credit_rule'], pred)}")
print(f"GINI: {roc_auc_score(df_test['credit_rule'], pred) * 2 - 1}")
df_test['pd'] = pred

[LightGBM] [Warning] min_data_in_leaf is set=800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=800
PRC_AUC: 0.9922050480473299
GINI: 0.2516728116891984


In [108]:
test_answer

,APPLICATIONID,pd
0,Д\286\011515162,0.964567
1,Д\286\011968491,0.954173
2,Д\286\011811529,0.977667
3,Д\286\011855826,1.000000
4,Д\286\011501724,1.000000
...,...,...
822,Д\286\011872662,0.983158
823,Д\286\011739541,0.977667
824,Д\286\012025070,0.954173
825,Д\286\011908721,0.982759


In [107]:
train_answer

,APPLICATIONID,pd
0,Д\286\011890015,0.954173
1,Д\286\011488424,0.950704
2,Д\286\011576373,0.809524
3,Д\286\011648144,0.954173
4,Д\286\011890015,0.500000
...,...,...
3299,Д\286\012038249,0.964567
3300,Д\286\012018301,0.961538
3301,Д\286\011908428,0.982759
3302,Д\286\012019546,0.977667


,APPLICATIONID,pd
0,Д\286\012242145,0.951923
1,Д\286\012269112,0.951923
2,Д\286\012232259,1.000000
3,Д\286\012073994,0.977667
4,Д\286\012121763,0.822222
...,...,...
2730,Д\286\012169335,0.977667
2731,Д\286\012084377,0.983158
2732,Д\286\012302965,0.954173
2733,Д\286\012282584,0.983158


In [111]:
main = pd.concat([df_test[['APPLICATIONID', 'pd']], train_answer, test_answer], ignore_index=True)

In [113]:
main.to_parquet("data_features/audio_pd.parquet")